In [13]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import json
import pandas as pd
import time
# from extracted_questions import Physics, Chemistry, Mathematics ,PhysicsA, ChemistryA, MathematicsA 
import os


In [14]:
physics_topics = ["Vectors","Types of Forces","Newton’s Laws of Motion","Friction","Basics of calculus","1 D Kinematics","2 D Kinematics","Circular Motion","Work, Power and Energy","Center of Mass","Conservation of Momentum","Rotation","Fluid Statics and Dynamics","Properties of Solids","Surface Tension and Viscosity","Simple Harmonic Motion","Combination of SHM","Damping and Resonance","Waves on a string","Sound Waves","Light Waves and Interference","Ray Optics","Thermal Expansion and Calorimetry","Kinetic Theory of Gases","1st Law of Thermodynamics & specific heats","Heat transfer","Electrostatics","Dielectrics and Capacitors","Current Electricity","Magnetism","Electro Magnetic Induction","Alternating Current","Modern Physics","Gravitation"]

maths_topics = ["Number System", "Factors and Indices","Sets","Progressions and Series","Quadratic Equations & Theory of Equations","Logarithms","Trigonometric Identities and Equations","Determinants and Matrices","Straight Lines and Pair of Lines","Circles","Permutations and Combinations","Binomial Theorem","Complex Numbers","Solution of Triangles","Functions","Inverse Trigonometry","Limits and Continuity","Derivatives and Method of Differentiation","Application of Derivatives","Indefinite Integration","Definite Integration","Area Under the Curves","Differential Equations","Vectors and 3D Geometry","Probability","Parabola","Ellipse","Hyperbola"
]

chemistry_topics =["Periodic Table","Chemical Bonding","Atomic Structure","Mole Concept","Gaseous State","Thermodynamics","Thermochemistry","Chemical Equilibrium","Ionic Equilibrium","Redox Reactions","s-block Elements","Hydrogen and Related Compounds","Nomenclature","Solid State","Boron Family","Structural and Geometrical Isomerism","Liquid Solutions and Colligative Properties","Chemical Kinetics","Surface Chemistry","Carbon Family","Metallurgy","d-block elements","f-block elements","General Organic Chemistry","Electrochemistry","Reaction Mechanisms","Tautomerism","Optical Isomerism","Alkanes","Alkenes","Alkynes","Arenes","Alkyl and Aryl Halides","Alcohols, Phenols and Ethers","Aldehydes and Ketones","Nitrogen Family","Amines","Carboxylic Acid and Derivatives","Qualitative Analysis","Coordination compounds","Biomolecules and Polymers","16, 17 and 18 family","Practical Organic Chemistry"
]
maths_questions = [    "Q.1 If y = f(x) makes positive intercepts of 2 and 0 units on the x and y axes respectively and encloses an area of 3/4 square units with the axes, then ∫₀² xf'(x)dx is (A) 3/2 (B) 1 (C) 5/4 (D) -3/4"]






In [15]:
accurate_prompt = strict_prompt = ChatPromptTemplate.from_template("""
SYSTEM:
You are a classification assistant. You MUST output ONLY a single valid JSON object and NOTHING else.
Use only the exact category names listed below (exact spelling). Do not invent or alter category names.

Categories:
{topics}

RULES (must follow exactly):
1) Output only ONE JSON object. No explanation, no backticks, no extra text.
2) Use ONLY keys from the categories above (exact spellings).
3) The numeric values must be intezers and their sum must be exactly 100.0.
4) If a question clearly belongs to a single category, give that category 100.0.
5) If ambiguous, split percentage accordingly but still sum to 100.0.
6) classify based on the answer, not just the question and if you cannot solve skip the question.
7) properly read and understand the question and answer before classifying.
8) strictly add only non zero results to the JSON object.



QUESTION: {questions}



OUTPUT (ONLY the valid JSON object):
""")

In [16]:
model1 = ChatOllama(model="llama2:70b", temperature=0)
model2 = ChatOllama(model="llama3.1:70b", temperature=0)
model3 = ChatOllama(model="llama-pro:8b-instruct-fp16", temperature=0)
model4 = ChatOllama(model="qwen2.5-coder:32b", temperature=0)
model5 = ChatOllama(model="qwen3:14b", temperature=0)
model6 = ChatOllama(model="gemma2:27b", temperature=0)  # Add or remove
models =[model3,model4,model5,model6]  # Add or remove

In [17]:
def get_topic_vector(question,topic_string,model):
    """Get the topic vector for a single question"""
    try:
        chain = accurate_prompt | model
        response = chain.invoke({"questions": question ,"topics":topic_string})
        response_text = response.content.strip()
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        json_str = response_text[json_start:json_end]       
        return json.loads(json_str)
    except Exception as e:
        print(f"Error: {e}")
        return {"Error": 100.0}
    

In [18]:
for i in range(1):
        all_results = []
        for model in models:
            print(f"Using model: {model}")
            
            for j, q in enumerate(maths_questions):
                print(f"Processing question {j+1}...")
                if j%10 == 0 and j != 0:
                    time.sleep(10)  # Pause for 2 seconds every 10 questions to avoid rate limits
                topic_data = get_topic_vector(q,maths_topics,model)
            
                all_results.append({
                    "question_id": j+1,
                    "question": q,
                    "topic_vector": topic_data
                })
                print(all_results[-1])

Using model: model='llama-pro:8b-instruct-fp16' temperature=0.0
Processing question 1...
{'question_id': 1, 'question': "Q.1 If y = f(x) makes positive intercepts of 2 and 0 units on the x and y axes respectively and encloses an area of 3/4 square units with the axes, then ∫₀² xf'(x)dx is (A) 3/2 (B) 1 (C) 5/4 (D) -3/4", 'topic_vector': {'Number System': 0, 'Factors and Indices': 0, 'Sets': 0, 'Progressions and Series': 0, 'Quadratic Equations & Theory of Equations': 0, 'Logarithms': 0, 'Trigonometric Identities and Equations': 0, 'Determinants and Matrices': 0, 'Straight Lines and Pair of Lines': 100.0, 'Circles': 0, 'Permutations and Combinations': 0, 'Binomial Theorem': 0, 'Complex Numbers': 0, 'Solution of Triangles': 0, 'Functions': 100.0, 'Inverse Trigonometry': 0, 'Limits and Continuity': 0, 'Derivatives and Method of Differentiation': 0, 'Application of Derivatives': 0, 'Indefinite Integration': 0, 'Definite Integration': 0, 'Area Under the Curves': 0, 'Differential Equations':